In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gym
from gym import spaces
import math
from pygame.math import Vector2
from sklearn.preprocessing import StandardScaler
import pygame

def distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    distance = math.sqrt(dx**2 + dy**2)
    return distance

def intersection(x1, y1, x2, y2, x3, y3, x4, y4):
    line1_start = Vector2(x1, y1)
    line1_end = Vector2(x2, y2)
    line2_start = Vector2(x3, y3)
    line2_end = Vector2(x4, y4)

    line1_direction = line1_end - line1_start
    line2_direction = line2_end - line2_start

    if line1_direction.cross(line2_direction) != 0:
        intersection_point = line1_start + line1_direction * ((line2_start - line1_start).cross(line2_direction) / line1_direction.cross(line2_direction))
        
        if (min(x1, x2) <= intersection_point.x <= max(x1, x2) and
            min(y1, y2) <= intersection_point.y <= max(y1, y2) and
            min(x3, x4) <= intersection_point.x <= max(x3, x4) and
            min(y3, y4) <= intersection_point.y <= max(y3, y4)):
            return intersection_point
            
    return None

def find_intersection_points(barrier_array, car_x, car_y, end_x, end_y):
    intersection_points = []
    for i, barrier in enumerate(barrier_array):
        intersection_point = intersection(barrier[0], barrier[1], barrier[2], barrier[3], car_x, car_y, end_x, end_y)
        if intersection_point:
            intersection_points.append(intersection_point)
    return intersection_points

def find_distance_to_intersection(intersection_points, car_x, car_y):
    closest_distance = float('inf')
    closest_intersection = None

    for point in intersection_points:
        x1, y1 = point
        
        dist = math.sqrt((x1 - car_x)**2 + (y1 - car_y)**2)

        if dist < closest_distance:
            closest_distance = dist
            closest_intersection = point

    return closest_distance, closest_intersection

class MyCarEnv(gym.Env):
    def __init__(self):
        self.action_space = spaces.Discrete(3)        

        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        self.GREEN = (0, 255, 0)
        self.RED = (255, 0, 0)

        self.barrier_array = np.array([[203, 270, 291, 151], [291, 151, 485, 181], [485, 181, 535, 349], [535, 349, 612, 367], [612, 367, 762, 226], 
                                               [762, 226, 962, 280], [962, 280, 980, 469], [980, 469, 934, 564], [934, 564, 739, 647], [739, 647, 641, 612],
                                                 [641, 612, 598, 455], [598, 455, 515, 432], [515, 432, 403, 528], [403, 528, 240, 501], [240, 501, 186, 390], 
                                                 [186, 390, 205, 264], [327, 238, 279, 335], [279, 335, 297, 405], [297, 405, 391, 424], [391, 424, 449, 368], 
                                                 [449, 368, 428, 274], [428, 274, 326, 239], [670, 421, 716, 550], [716, 550, 848, 504], [848, 504, 885, 438], 
                                                 [885, 438, 879, 350], [879, 350, 784, 312], [784, 312, 670, 421]])

        self.reward_lines_array = np.array([[428, 274, 486, 181], [328, 239, 290, 153], [278, 333, 203, 267], [280, 337, 186, 391], [296, 406, 239, 500], 
                                            [390, 426, 402, 527], [449, 367, 517, 433], [449, 368, 537, 348], [599, 456, 669, 421], [669, 421, 611, 367], 
                                            [784, 312, 762, 228], [879, 350, 962, 280], [886, 438, 982, 471], [846, 506, 933, 563], [716, 549, 739, 645], [717, 550, 640, 611]])

        self.start_x = 560
        self.start_y = 404
        self.start_angle = 190

        self.car_x = self.start_x 
        self.car_y = self.start_y
        self.car_angle = self.start_angle
        self.speed = 15
        self.rotation_speed = 10

        self.num_rays = 12
        self.ray_length = 1000

        
        self.WINDOW_WIDTH, self.WINDOW_HEIGHT = 1268, 840

        self.car_color = self.BLACK
        self.car_width = 20
        self.car_height = 20

        self.state = np.ones(self.num_rays, dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(self.num_rays,))

        self.reward = 0
        self.car_surface = pygame.Surface((self.car_width, self.car_height), pygame.SRCALPHA)
        
        self.distance_array_for_fit = np.ones((1, self.num_rays), dtype=np.float32)

        self.reward_last = []



    def reset(self):

        self.car_x = self.start_x 
        self.car_y = self.start_y
        self.car_angle = self.start_angle
        self.num_steps = 0
        self.reward = 0
        self.reward_last = []
        self.reward_lines_array = np.array([[428, 274, 486, 181], [328, 239, 290, 153], [278, 333, 203, 267], [280, 337, 186, 391], [296, 406, 239, 500], 
                                [390, 426, 402, 527], [449, 367, 517, 433], [449, 368, 537, 348], [599, 456, 669, 421], [669, 421, 611, 367], 
                                [784, 312, 762, 228], [879, 350, 962, 280], [886, 438, 982, 471], [846, 506, 933, 563], [716, 549, 739, 645], [717, 550, 640, 611]])
        
        self.barrier_array = np.array([[203, 270, 291, 151], [291, 151, 485, 181], [485, 181, 535, 349], [535, 349, 612, 367], [612, 367, 762, 226], 
                                        [762, 226, 962, 280], [962, 280, 980, 469], [980, 469, 934, 564], [934, 564, 739, 647], [739, 647, 641, 612],
                                        [641, 612, 598, 455], [598, 455, 515, 432], [515, 432, 403, 528], [403, 528, 240, 501], [240, 501, 186, 390], 
                                        [186, 390, 205, 264], [327, 238, 279, 335], [279, 335, 297, 405], [297, 405, 391, 424], [391, 424, 449, 368], 
                                        [449, 368, 428, 274], [428, 274, 326, 239], [670, 421, 716, 550], [716, 550, 848, 504], [848, 504, 885, 438], 
                                        [885, 438, 879, 350], [879, 350, 784, 312], [784, 312, 670, 421]])
        return self.state

    def step(self, action):   
        done = False

        self.reward = 0
        self.num_steps += 1
        if action==0:
            self.car_angle += self.rotation_speed  
        elif action==2:
            self.car_angle -= self.rotation_speed 

        if len(self.reward_lines_array)>1:
            for index, line in enumerate(self.reward_lines_array):

                x1, y1, x2, y2 = line
                if  ((intersection(x1, y1, x2, y2, self.car_x-10, self.car_y-10, self.car_x+10, self.car_y+10)) or 
                                                    (intersection(x1, y1, x2, y2, self.car_x-10, self.car_y+10, self.car_x+10, self.car_y-10))):
                    if np.any(self.reward_last):
                        if not np.array_equal(np.array(line), self.reward_last):
                            self.reward = 1
                            self.barrier_array = np.append(self.barrier_array, self.reward_lines_array[np.all(self.reward_lines_array == self.reward_last, axis=1)], axis=0)
                            self.reward_lines_array = self.reward_lines_array[~np.all(self.reward_lines_array == self.reward_last, axis=1)]
                    else:
                        self.reward = 1
                    self.reward_last = np.array(line)
                    

        else:
            self.reward_lines_array = np.array([[428, 274, 486, 181], [328, 239, 290, 153], [278, 333, 203, 267], [280, 337, 186, 391], [296, 406, 239, 500], 
                                            [390, 426, 402, 527], [449, 367, 517, 433], [449, 368, 537, 348], [599, 456, 669, 421], [669, 421, 611, 367], 
                                            [784, 312, 762, 228], [879, 350, 962, 280], [886, 438, 982, 471], [846, 506, 933, 563], [716, 549, 739, 645], [717, 550, 640, 611]])
            
            self.barrier_array = np.array([[203, 270, 291, 151], [291, 151, 485, 181], [485, 181, 535, 349], [535, 349, 612, 367], [612, 367, 762, 226], 
                                            [762, 226, 962, 280], [962, 280, 980, 469], [980, 469, 934, 564], [934, 564, 739, 647], [739, 647, 641, 612],
                                            [641, 612, 598, 455], [598, 455, 515, 432], [515, 432, 403, 528], [403, 528, 240, 501], [240, 501, 186, 390], 
                                            [186, 390, 205, 264], [327, 238, 279, 335], [279, 335, 297, 405], [297, 405, 391, 424], [391, 424, 449, 368], 
                                            [449, 368, 428, 274], [428, 274, 326, 239], [670, 421, 716, 550], [716, 550, 848, 504], [848, 504, 885, 438], 
                                            [885, 438, 879, 350], [879, 350, 784, 312], [784, 312, 670, 421]])

        for line in self.barrier_array:
            x1, y1, x2, y2 = line
            if (intersection(x1, y1, x2, y2, self.car_x-10, self.car_y-10, self.car_x+10, self.car_y+10)) or (intersection(x1, y1, x2, y2, self.car_x-10, self.car_y+10, self.car_x+10, self.car_y-10)):
                self.reward += -5
                done = True

        radians = math.radians(self.car_angle)
        position = Vector2(self.car_x, self.car_y)
        direction = Vector2(math.cos(radians), -math.sin(radians))
        position += direction * self.speed
        self.car_x, self.car_y = position.x, position.y

        end_xy = np.zeros((self.num_rays, 2))


        for i in range(self.num_rays):
            for j in range(2):
                if j==1:
                    end_xy[i][j] = self.car_y - self.ray_length * math.sin(math.radians( self.car_angle+i*360/ self.num_rays))
                else:
                    end_xy[i][j] = self.car_x + self.ray_length * math.cos(math.radians( self.car_angle+i*360/ self.num_rays))

        self.line_intersection_xy = []
        for end_x,end_y in end_xy:
            self.line_intersection_xy.append(find_intersection_points( self.barrier_array, self.car_x, self.car_y, end_x, end_y))

        distance_line_array = []
        for i in range(len(self.line_intersection_xy)):
            distance, closest_intersection = find_distance_to_intersection(self.line_intersection_xy[i], self.car_x, self.car_y)
            if closest_intersection is not None and not math.isinf(distance):
                distance_line_array.append(distance)
            else:
                distance_line_array.append(1000)
                # self.reward = -10
                # done = True
        
        # self.car_xy = np.array([int(self.car_x), int(self.car_y)])

        # self.state = np.concatenate([self.car_xy, distance_line_array])
        distance_line_array = np.array(distance_line_array)
        self.distance_array_for_fit = np.vstack((distance_line_array, self.distance_array_for_fit))[:500]
   

        scaler.fit(self.distance_array_for_fit)
        scaled_distance_line_array = scaler.transform(distance_line_array.reshape(1, -1))

        self.state = scaled_distance_line_array.flatten()
        
        return self.state, self.reward, done

    def render(self, mode='human'):

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        pygame.draw.circle(self.car_surface, self.car_color, (20,20), self.car_width)

        clock = pygame.time.Clock()
        self.screen.fill(self.WHITE) 

        pygame.draw.line(self.screen, self.BLACK, (self.car_x-10, self.car_y-10), (self.car_x+10, self.car_y+10), 5)
        pygame.draw.line(self.screen, self.BLACK, (self.car_x-10, self.car_y+10), (self.car_x+10, self.car_y-10), 5)
        for line in self.barrier_array:
            x1, y1, x2, y2 = line
            pygame.draw.line(self.screen, self.BLACK, (x1, y1), (x2, y2))


        for line in self.reward_lines_array:
            x1, y1, x2, y2 = line
            pygame.draw.line(self.screen, self.GREEN, (x1, y1), (x2, y2))

        pygame.draw.circle(self.screen, self.BLACK, (self.car_x, self.car_y), (10))

        for i, intersection_points in enumerate(self.line_intersection_xy):
            closest_distance, closest_intersection = find_distance_to_intersection(intersection_points, self.car_x, self.car_y)
            if closest_intersection:
                if i == 0:
                    pygame.draw.line(self.screen, self.BLACK, (self.car_x, self.car_y), (int(closest_intersection[0]), int(closest_intersection[1])), 5)
                else:
                    pygame.draw.line(self.screen, self.BLACK, (self.car_x, self.car_y), (int(closest_intersection[0]), int(closest_intersection[1])), 1)
                pygame.draw.circle(self.screen, self.RED, (int(closest_intersection[0]), int(closest_intersection[1])), 5)
        pygame.display.flip()


        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.car_angle += self.rotation_speed  
        if keys[pygame.K_RIGHT]:
            self.car_angle -= self.rotation_speed  
        if keys[pygame.K_ESCAPE]:
            pygame.quit()
        
        clock.tick(120)

env = MyCarEnv()



class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1).to(device)
        # self.fc2 = nn.Linear(hidden_size1, hidden_size2).to(device)
        # self.fc3 = nn.Linear(hidden_size2, hidden_size3).to(device)
        self.fc4 = nn.Linear(hidden_size1, output_size).to(device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x)).to(device)
        x = torch.softmax(self.fc4(x), dim=-1).to(device)
        return x

class PolicyNetwork1(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(PolicyNetwork1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1).to(device)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2).to(device)
        self.fc3 = nn.Linear(hidden_size2, output_size).to(device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x)).to(device)
        x = torch.relu(self.fc2(x)).to(device)
        x = torch.softmax(self.fc3(x), dim=-1).to(device)
        return x
    
class PolicyNetwork2(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(PolicyNetwork2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1).to(device)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2).to(device)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3).to(device)
        self.fc4 = nn.Linear(hidden_size3, output_size).to(device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x)).to(device)
        x = torch.relu(self.fc2(x)).to(device)
        x = torch.relu(self.fc3(x)).to(device)
        x = torch.softmax(self.fc4(x), dim=-1).to(device)
        return x


def select_action(policy_net, state):
    state = torch.tensor(state, dtype=torch.float32)
    action_probs = policy_net(state)
    action = np.random.choice(len(action_probs.detach().numpy()), p=action_probs.detach().numpy())
    return action


def train(policy_net, optimizer, episodes, max_reward, hidden_size1, hidden_size2, hidden_size3, learning_rate):
    for episode in range(episodes):
        state = env.reset()
        episode_states = []
        episode_actions = []
        episode_rewards = []
        steps = 0
        while True:
            action = select_action(policy_net, state)
            next_state, reward, done = env.step(action)
            steps += 1
            if render:
                env.render()

            episode_states.append(state)
            episode_actions.append(action)
            episode_rewards.append(reward)

           
        
            state = next_state
            if steps >= 1000000:
                torch.save(policy_net, f'net/tarakan1_{max_reward+5}_{hidden_size1}_{hidden_size2}_{hidden_size3}_{learning_rate}.pth')
                print(f"Tarakan1 Ultra Episodes {episode}:{hidden_size1}_{hidden_size2}_{hidden_size3}_{learning_rate}  SAVED!")
                done = True
            if done:
                break
        returns = []
        R = 0
        for r in episode_rewards[::-1]:
            R = r + gamma * R
            returns.insert(0, R)

        policy_loss = []
        for i in range(len(episode_states)):
            action = episode_actions[i]
            G = returns[i]
            state = episode_states[i]
            action_prob = policy_net(torch.tensor(state, dtype=torch.float32))[action]
            policy_loss.append(-torch.log(action_prob) * G)

        optimizer.zero_grad()
        policy_loss = torch.stack(policy_loss).sum()

        policy_loss.backward()
        optimizer.step()
        if sum(episode_rewards) > max_reward:
            max_reward = sum(episode_rewards)
            torch.save(policy_net, f'net/tarakan1_{max_reward+5}_{hidden_size1}_{hidden_size2}_{hidden_size3}_{learning_rate}.pth')
            print(f"Episode {episode}: Max Reward = {max_reward+5} SAVED!")
        print(f"Episode {episode}: Total Reward = {sum(episode_rewards)+5} Max Reward = {max_reward+5}, hidden_size1 = {hidden_size1}, hidden_size2 = {hidden_size2}, hidden_size3 = {hidden_size3}, леарнинг рейт = {learning_rate}")
    return max_reward




def test(policy_net, num_episodes=10):
    total_reward = -10
    for episode in range(num_episodes):
        
        state = env.reset()

        while True:
            action = select_action(policy_net, state)
            next_state, reward, done = env.step(action)

            total_reward += reward

            if render:
                env.render()

            if done:
                print(f"Episode {episode + 1}: Total Reward = {total_reward}")
                break

            state = next_state

    env.close()
    return total_reward


scaler = StandardScaler()

render = False

if render:
    env.screen = pygame.display.set_mode((env.WINDOW_WIDTH, env.WINDOW_HEIGHT))

device = torch.device("cpu")

gamma = 0.99

episodes = 1000


results = []
max_reward = -10

In [ ]:
for i in range(22, 100):
    input_size = env.num_rays
    output_size = 3
    hidden_size1 = i
    learning_rate = 0.01

    policy_net = PolicyNetwork(input_size, hidden_size1, output_size).to(device)
    optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
    max_reward = train(policy_net, optimizer, episodes, max_reward, hidden_size1, 0, 0, learning_rate)

    restest = (f'1 скрытый слой, hidden_size1 = {hidden_size1}, леарнинг рейт = {learning_rate}, счет = {test(policy_net, num_episodes=10)}')
    print(restest)
    results.append(restest)

In [ ]:
episodes=10000
for i in range(1, 100):
    input_size = env.num_rays
    output_size = 3
    hidden_size1 = i
    learning_rate = 0.001

    policy_net = PolicyNetwork(input_size, hidden_size1, output_size).to(device)
    optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
    max_reward = train(policy_net, optimizer, episodes, max_reward, hidden_size1, 0, 0, learning_rate)

    restest = (f'1 скрытый слой, hidden_size1 = {hidden_size1}, леарнинг рейт = {learning_rate}, счет = {test(policy_net, num_episodes=10)}')
    print(restest)
    results.append(restest)

In [ ]:
for i in range(1, 100):
    for a in range(1, 100):
        input_size = env.num_rays
        output_size = 3
        hidden_size1 = i
        hidden_size2 = a
        learning_rate = 0.001
        
        policy_net = PolicyNetwork1(input_size, hidden_size1, hidden_size2, output_size).to(device)
        optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)

        max_reward = train(policy_net, optimizer, episodes, max_reward, hidden_size1, hidden_size2, 0, learning_rate)
        restest = (f'2 скрытых слоя, hidden_size1 = {hidden_size1}, hidden_size2 = {hidden_size2},леарнинг рейт = {learning_rate}, счет = {test(policy_net, num_episodes=10)}')
        print(restest)
        results.append(restest)




In [ ]:

input_size = env.num_rays
output_size = 3
hidden_size1 = 35
learning_rate = 0.01

policy_net = PolicyNetwork(input_size, hidden_size1, output_size).to(device)
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
max_reward = train(policy_net, optimizer, episodes, max_reward, hidden_size1, 0, 0, learning_rate)

restest = (f'1 скрытый слой, hidden_size1 = {hidden_size1}, леарнинг рейт = {learning_rate}, счет = {test(policy_net, num_episodes=10)}')
print(restest)
results.append(restest)

In [2]:
render = True
env.screen = pygame.display.set_mode((env.WINDOW_WIDTH, env.WINDOW_HEIGHT))
policy_net = torch.load(f'net/tarakan1/6_0_0_0.0001_442.pth')
print(test(policy_net, num_episodes=100))

Episode 1: Total Reward = -14
Episode 2: Total Reward = -17
Episode 3: Total Reward = -16
Episode 4: Total Reward = -19
Episode 5: Total Reward = -21
Episode 6: Total Reward = -20
Episode 7: Total Reward = -18
Episode 8: Total Reward = -15
Episode 9: Total Reward = -3
Episode 10: Total Reward = 7
Episode 11: Total Reward = 9
Episode 12: Total Reward = 10
Episode 13: Total Reward = 10
Episode 14: Total Reward = 12
Episode 15: Total Reward = 14
Episode 16: Total Reward = 41
Episode 17: Total Reward = 76
Episode 18: Total Reward = 118
Episode 19: Total Reward = 122
Episode 20: Total Reward = 145
Episode 21: Total Reward = 187
Episode 22: Total Reward = 207
Episode 23: Total Reward = 219
Episode 24: Total Reward = 222
Episode 25: Total Reward = 344
Episode 26: Total Reward = 348
Episode 27: Total Reward = 351
Episode 28: Total Reward = 393
Episode 29: Total Reward = 397
Episode 30: Total Reward = 401
Episode 31: Total Reward = 412
Episode 32: Total Reward = 422
Episode 33: Total Reward = 5

error: video system not initialized

In [ ]:
torch.save(policy_net, f'net/tarakan_ultra.pth')